In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import myLogFun
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

In [2]:
# all data with cluster id import
clustered_df = pd.read_csv('../files/cluster_df.csv')
clustered_df_log,log_tran_col = myLogFun.log_tran(clustered_df.copy(),['Total Traffic Volume (MB)','cluster_id'])

In [3]:
# filter only selected cluster
df1 = clustered_df_log[clustered_df_log['cluster_id']==1].reset_index(drop=True).drop('cluster_id',axis=1)

# Making pipeline for the model

In [11]:
X1 = df1.iloc[:,:-1].values
y1 = df1.iloc[:,-1].values

X1_train,X1_test,y1_train,y1_test=train_test_split(X1,y1,test_size=0.3, random_state=42)

In [4]:
steps = [('scaler',StandardScaler()),('reg',lgb.LGBMRegressor())]

In [5]:
pipeline = Pipeline(steps)

In [35]:
params = dict(
        reg__boosting_type=['gbdt'],
        reg__n_estimators=[2000],
        reg__learning_rate=[0.01,0.05,0.1],
        reg__num_leaves=[30,50,70],
)

In [36]:
cv = GridSearchCV(pipeline, param_grid=params,cv=5)

In [37]:
cv.fit(X1_train,y1_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('reg', LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_spli...0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'reg__boosting_type': ['gbdt'], 'reg__n_estimators': [2000], 'reg__learning_rate': [0.01, 0.05, 0.1], 'reg__num_leaves': [30, 50, 70]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [38]:
y_pred=cv.predict(X1_test)

In [40]:
r2_score(y1_test,y_pred)

0.7060718536795221